In [28]:
import sys
import pyspark.sql
from pyspark.sql import functions as F
from pyspark.sql.types import BooleanType

In [2]:
spark = SparkSession\
        .builder\
        .appName("Modulo 3 Desafio")\
        .enableHiveSupport()\
        .getOrCreate()

24/06/01 14:19:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
cnaes = '/home/telma/xpe/engenharia_dados/dados/cnaes/cnaes.csv'
estab1 = '/home/telma/xpe/engenharia_dados/dados/estabelecimentos/estabelecimentos/estabelecimentos-1.csv'
estab2 = '/home/telma/xpe/engenharia_dados/dados/estabelecimentos/estabelecimentos/estabelecimentos-2.csv'
estab3 = '/home/telma/xpe/engenharia_dados/dados/estabelecimentos/estabelecimentos/estabelecimentos-3.csv'

24/06/01 14:19:59 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
df_cnae = spark.read.csv(cnaes, header=True, inferSchema=True, sep=';')

In [5]:
df_estab1 = spark.read.csv(estab1, header=True, inferSchema=True, sep=';')

In [8]:
df_estab2 = spark.read.csv(estab2, header=True, inferSchema=True, sep=';')

In [9]:
df_estab3 = spark.read.csv(estab3, header=True, inferSchema=True, sep=';')

In [10]:
df_estab = df_estab1.union(df_estab2.union(df_estab3))

In [19]:
df_estab.count()

20996744

In [12]:
df_estab.createOrReplaceTempView("estab")

### 1) Quantos estabelecimentos existem?

In [13]:

spark.sql("""select count(distinct CNPJ_BASICO,CNPJ_ORDEM,CNPJ_DV) from estab """).show()

24/06/01 14:34:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 14:34:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 14:34:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 14:34:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 14:34:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 14:34:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 14:34:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+------------------------------------------------+
|count(DISTINCT CNPJ_BASICO, CNPJ_ORDEM, CNPJ_DV)|
+------------------------------------------------+
|                                        20996744|
+------------------------------------------------+



### 2) Quantas colunas existem na tabela estabelecimentos e quantas são identificadas como números?

In [15]:
df_estab.printSchema()

root
 |-- CNPJ_BASICO: integer (nullable = true)
 |-- CNPJ_ORDEM: integer (nullable = true)
 |-- CNPJ_DV: integer (nullable = true)
 |-- IDENTIFICADOR_MATRIZ_FILIAL: integer (nullable = true)
 |-- NOME_FANTASIA: string (nullable = true)
 |-- SITUACAO_CADASTRAL: integer (nullable = true)
 |-- DATA_SITUACAO_CADASTRAL: integer (nullable = true)
 |-- MOTIVO_SITUACAO_CADASTRAL: integer (nullable = true)
 |-- NOME_DA_CIDADE_NO_EXTERIOR: string (nullable = true)
 |-- PAIS: integer (nullable = true)
 |-- DATA_INICIO_ATIVIDADE: integer (nullable = true)
 |-- CNAE_PRINCIPAL: integer (nullable = true)
 |-- CNAE_SECUNDARIA: string (nullable = true)
 |-- TIPO_LOGRADOURO: string (nullable = true)
 |-- LOGRADOURO: string (nullable = true)
 |-- NUMERO: string (nullable = true)
 |-- COMPLEMENTO: string (nullable = true)
 |-- BAIRRO: string (nullable = true)
 |-- CEP: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- MUNICIPIO: integer (nullable = true)
 |-- DDD_1: string (nullable = true)

### 3) Gere arquivo parquet a partir de "estabelecimentos" e compare o tamanho dos arquivos parquet e csv originais.

In [16]:
df_estab.write\
        .parquet('/home/telma/xpe/engenharia_dados/dados/estabelecimentos/estabelecimentos/estabelecimentos.parquet',
                mode = 'overwrite')

### 4) Spark.sql ==>> Quantos estabelecimentos não tem logradouro informado.

In [18]:
spark.sql("select count(*) from estab where logradouro is null").show()

+--------+
|count(1)|
+--------+
|     828|
+--------+



### 5) Quantos estabelecimentos ficam localizados numa avenida?

In [25]:
def is_avenida(logradouro):
    if str(logradouro).upper().startswith('AVENIDA'):
        return True
    else:
        return False

In [32]:
# is_avenidaUDF = F.udf(lambda logradouro: is_avenida(logradouro), BooleanType())
spark.udf.register("is_avenidaUDF", is_avenida, BooleanType())

<function __main__.is_avenida(logradouro)>

In [33]:
spark.sql("""
          select count(*) from estab where is_avenidaUDF(logradouro) = True
          """).show()

+--------+
|count(1)|
+--------+
|   52587|
+--------+



### 6) Quantos CEP's distintos existem em estabelecimentos?

In [37]:
###  Resposta 958396 não existe

# spark.sql("select count(distinct CEP) from estab").show()
df_estab.agg(F.countDistinct(df_estab.CEP)).show()

+-------------------+
|count(DISTINCT CEP)|
+-------------------+
|             958396|
+-------------------+



### 7) Quantos CNAEs existem na tabela de CNAEs

In [38]:
df_cnae.createOrReplaceTempView("cnae")

In [40]:
spark.sql("select count(distinct cnae) from cnae").show()

+--------------------+
|count(DISTINCT cnae)|
+--------------------+
|                1359|
+--------------------+



### 8) Quantos estabelecimentos estão relacionados a cultivo?

In [45]:
def is_cnae_cultivo(desc_cnae):
    if str(desc_cnae).upper().startswith("CULTIVO"):
        return True
    else:
        return False

In [46]:
spark.udf.register("is_cnae_cultivoUDF", is_cnae_cultivo, BooleanType())

24/06/01 16:02:45 WARN SimpleFunctionRegistry: The function is_cnae_cultivoudf replaced a previously registered function.


<function __main__.is_cnae_cultivo(desc_cnae)>

In [49]:
spark.sql("""
select count(*) from estab e inner join cnae c 
    on e.CNAE_PRINCIPAL = c.CNAE 
  --  OR e.CNAE_SECUNDARIA = c.CNAE
WHERE is_cnae_cultivoUDF(c.DESCRICAO_CNAE) = True
""").show()

+--------+
|count(1)|
+--------+
|  200243|
+--------+



### 11) Quantos estabelecimentos são filiais?

In [50]:
spark.sql("select count(distinct CNPJ_BASICO, CNPJ_ORDEM, CNPJ_DV) from estab where IDENTIFICADOR_MATRIZ_FILIAL =  2").show()

+------------------------------------------------+
|count(DISTINCT CNPJ_BASICO, CNPJ_ORDEM, CNPJ_DV)|
+------------------------------------------------+
|                                         1093082|
+------------------------------------------------+

